# Double-Bracket Iteration Strategy: magnetic field (onsite Z)
This notebook shows the diagonalization process of DBI using the magnetic field strategy, which varies the diagonal operator $D$ by gradient descent. To find the gradient with respect to $D$, parameterization of $D$ is required. For the purpose of this notebook, we represent it by onsite Pauli-Z operators, i.e.

$$ D = \sum \alpha_i Z_i $$

Note that it is also possible to have higher-order terms, such as $ D = \sum \alpha_i Z_i + \sum \beta_{i,j}Z_iZ_j+...$

The gradients are calculated under the premise that the diagonalization gain curve can be fitted by a polynomial, and that the iteration step duration is taken at the first dip of the curve.

In [ ]:
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend
from qibo.hamiltonians import Hamiltonian, SymbolicHamiltonian
from qibo.quantum_info import random_hermitian
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration
from qibo.models.dbi.utils import *

In [ ]:
def visualize_matrix(matrix, title=""):
    """Visualize hamiltonian in a heatmap form."""
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_title(title)
    try:
        im = ax.imshow(np.absolute(matrix), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute(matrix.get()), cmap="inferno")
    fig.colorbar(im, ax=ax)


## Test on random Hamiltonian

In [ ]:
# backend
set_backend("qibojit", "numba")
# initialize dbi object
nqubits = 5
h0 = random_hermitian(2**nqubits, seed=2)
dbi = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0))
print("Initial off diagonal norm", dbi.off_diagonal_norm)
visualize_matrix(dbi.h.matrix, title=f'Random hamiltonian with L={nqubits}')

In [ ]:
# generate the onsite Z operators
onsite_Z_ops = generate_onsite_Z_ops(nqubits)
d_coef = onsite_Z_decomposition(dbi.h.matrix, onsite_Z_ops)
d = sum([d_coef[i] * onsite_Z_ops[i] for i in range(nqubits)])
grad, s = gradient_onsite_Z(dbi,d,3, onsite_Z_ops)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)
print('s:', s)

In [ ]:
iters = 30
off_diagonal_norm_tot = [dbi.off_diagonal_norm]
s_step = [0]
for i in range(iters):
    s, d_coef, d = gradient_descent_onsite_Z(dbi, d_coef, d, onsite_Z_ops=onsite_Z_ops, max_evals=100)
    off_diagonal_norm_tot.append(dbi.off_diagonal_norm)
    s_step.append(s)

In [ ]:
plt.title(str(nqubits) + ' spins magnetic field diagonalization')
plt.plot(off_diagonal_norm_tot)
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

## Test on TFIM
Here we choose to customize our TFIM in the X axis using `SymbolicHamiltonian`. It is also possible to use Hadamard gate to rotate the TFIM inbuilt in `qibo`.

$$ H = -(\sum X_i X_{i+1} + \sum hZ_i)

In [ ]:
# generate the Hamiltonian
nqubits = 5
h = 1
H_TFIM = SymbolicHamiltonian( - h*symbols.Z(nqubits-1), nqubits=nqubits)
# add linear interaction terms
for i in range(nqubits-1):
    H_TFIM -= SymbolicHamiltonian(symbols.X(i)*symbols.X(i+1) + h*symbols.Z(i), nqubits=nqubits)
H_TFIM = H_TFIM.dense
visualize_matrix(H_TFIM.matrix, title=f'TFIM with L={nqubits} h={h}')

In [ ]:
# backend
set_backend("qibojit", "numba")
# initialize dbi object
dbi_TFIM = DoubleBracketIteration(deepcopy(H_TFIM))

In [ ]:
# generate the onsite Z operators
onsite_Z_ops = generate_onsite_Z_ops(nqubits)
d_coef = onsite_Z_decomposition(dbi_TFIM.h.matrix, onsite_Z_ops)
d = sum([d_coef[i] * onsite_Z_ops[i] for i in range(nqubits)])
grad, s = gradient_onsite_Z(dbi_TFIM,d,3, onsite_Z_ops)
print('Initial off-diagonal norm:', dbi.off_diagonal_norm)
print('The initial D coefficients:', d_coef)
print('Gradient:', grad)
print('s:', s)

In [ ]:
NSTEPS = 30
off_diagonal_norm_tot = [dbi_TFIM.off_diagonal_norm]
s_step = [0]
for _ in range(NSTEPS):
    s, d_coef, d = gradient_descent_onsite_Z(dbi_TFIM, d_coef, d, onsite_Z_ops=onsite_Z_ops, max_evals=100)
    off_diagonal_norm_tot.append(dbi_TFIM.off_diagonal_norm)
    s_step.append(s)
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {s} with d_coef {d_coef}, loss {dbi_TFIM.off_diagonal_norm}")

In [ ]:
plt.title(str(nqubits) + ' spins TFIM magnetic field diagonalization')
plt.plot(off_diagonal_norm_tot)
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In [ ]:
# the final matrix
visualize_matrix(dbi_TFIM.h.matrix)

## Different initial `d`
Next, we show the effect of different choices of the initial direction of the gradient descent method.